# Language Knowledge (Vocabulary)
Duration: 30 minutes
Content: This section tests your knowledge of Japanese vocabulary, including kanji readings, orthography, word formation, contextually-defined expressions, paraphrases, and usage
It mainly composes following five categories:
- ``Reading Kana`` (Pronunciation Questions): Given a kanji word, choose the correct kana reading.
- `Writing Kanji` (Writing Questions): Given a word written in kana, choose the correct kanji representation.
- `Word Meaning` Selection (Vocabulary Understanding): Choose the most suitable word to fill in the sentence from four options.
- `Synonym Replacement`: Select a word that has the same or similar meaning as the underlined word.
- `Vocabulary Usage`: Assess the usage of words in actual contexts, choosing the most appropriate word usage, including some common Japanese expressions or fixed phrases.

In [73]:
import pandas as pd
import json
import os
import random
import pickle
import re
import uuid
from typing import *
from langchain_openai import AzureOpenAI,AzureChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from dotenv import load_dotenv
from langchain_aws import ChatBedrock
from langchain.embeddings.base import Embeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain_community.embeddings import XinferenceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from typing import Annotated, Literal, Sequence
from typing_extensions import TypedDict
from IPython.display import display, Markdown, Latex
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from typing import Annotated, Sequence
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage,RemoveMessage,HumanMessage,AIMessage,ToolMessage
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List, Optional
from langchain_tavily import TavilySearch
from langchain.schema import Document
from langgraph.prebuilt import create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults
load_dotenv()

True

In [74]:
azure_llm = AzureChatOpenAI(
    azure_endpoint="https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview",
    api_key=os.environ["AZURE_API_KEY"],
    model_name="gpt-4o",
    api_version="2025-01-01-preview",
    temperature=0.5,
)

aws_llm = ChatBedrock(
    # model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
     model_id="us.anthropic.claude-3-5-haiku-20241022-v1:0",
    model_kwargs=dict(temperature=0.5),
    region = "us-east-2",
    aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"],
)

In [75]:
# Import N3 Vocabulary
file_path = '../../Vocab/n3.csv'
# Read the CSV file
data = pd.read_csv(file_path)
words = data.iloc[:, :2].sample(frac=1).reset_index(drop=True)
# Display the content of the CSV file
words.head()
vocab_dict = words.set_index(words.columns[0])[words.columns[1]].to_dict()
vocab_dict = json.dumps(vocab_dict, ensure_ascii=False, separators=(',', ':'))

#### load Models

#### Exam Paper Outline
### A. overall thinking the structure of an exam
1. distribution of the difficulty 
2. topics
3. reasoning

## Data Strcuture

# Kanji 读假名（读音问题）

In [76]:
def online_search(state):
    """
    Web search based on the re-phrased question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with appended web results
    """
    
    print("---WEB SEARCH---")
    
    topic = state['messages'][0].content
    
    tavily_search_tool = TavilySearch(
        max_results=5,
        topic="news",
        days=1
    )
    # Web search
    docs = tavily_search_tool.invoke({"query": topic})
    
    print(docs)

    web_results = "\n".join([d["content"] for d in docs["results"]])
    
    print("Web results: ", web_results)

    return {"documents": web_results, "topic": topic}

In [77]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display


# Graph state
class QuestionState(TypedDict):
    topic: str
    question: str
    documents: str
    messages: Annotated[list, add_messages]


example = """
問題5  
  
1番 正解: 1  
会話内容:
- 女: 足、痛そうだね。午後のテニスの練習、休んだら？  
- 男: そうする。今日は帰るね。  
- 女: 今日は練習、ないんだね。  
- 男: テニス、今日は休むの?  
  
---  
  
2番 正解: 2  
会話内容: 
- 男: 町の花火大会、今年はやらないことになったそうだよ。  
- 女: やらないかもしれないんだね。  
- 男: え? なんで? 楽しみにしてたのに・・・  
- 女: じゃ、見に行かなくきゃね。  
  
---  
  
3番 正解: 1  
会話内容:
- 男: 吉田さん、今回の旅行、楽しかったよ。吉田さんが案内してくれたおかげだよ。  
  1. 喜んでもらえてよかった  
  2. 一緒に行けなくてごめんね  
  3. 案内してくれてありがとう  

---  
   
#### 4番 正解: 1  
**会話内容:**  
- 男: 来週の食事会、参加できるかまだわからなくて、いつまでにお返事すればいいですか?  
  1. 今週中なら大丈夫ですよ  
  2. 参加できそうでよかったです  
  3. はい、返事お待ちしていますね  
  
---  
  
5番 正解: 3  
会話内容:
- 女: 曇ってきたね。雨が降らないうちに帰ろうか。  
  1. え? もう降ってきた?  
  2. 雨が止んでから帰るの?  
  3. 降る前に帰ったほうがいいね  
  
---  
  
6番 正解: 3  
会話内容:  
- 女: 森さん、悪いけど、ドアの近くにあるダンボール箱、倉庫に運んでくれる?  
  1. 倉庫にあるんですね。取ってきます  
  2. ありがとうございます。お願いします  
  3. あとでいいですか?  
  
---  
  
7番 正解: 1  
会話内容:
- 女: あの、こちらのお店、店の中の写真を撮っても構いませんか? すごく素敵なので。  
  1. あ、写真はご遠慮ください  
  2. 素敵な写真、ありがとうございます  
  3. 写真は撮ってなんないですよ  
  
---  
   
8番 正解: 2  
会話内容:
- 男: 今、課長から電話があったんですが、訪問先から会社に戻らずに帰宅されるそうです。  
  1. 一度会社に戻って来られるんですね  
  2. あ、そのまま家に帰られるんですね  
  3. え? 家に寄って来られるんですか?  
  
---  
  
9番 正解: 3  
会話内容:
- 男: 工事、遅れてるんだって? 課長に報告したほうがいいんじゃない?  
  1. 遅れてるって課長が言ってたんですか?  
  2. じゃ、報告はしないことにします  
  3. そうですね。伝えておきます  
"""

# Nodes
def question_draft_generator(state: QuestionState):
    """First LLM call to generate initial question"""
    print("---Generator----")
        
    search_result = state['documents'],
    
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """
                    You are a Japanese teacher. Your job is to write a listening question for candidates to prepare the original text and options for the listening dialogue based on the reference format. Instant response, students need to listen to the conversation, choose the option that matches the meaning of the question based on the listening content, select the appropriate answer for this sentence in the current context, and provide three options. After listening to a conversation, I often ask someone in the conversation what they are going to do next. Only refer to the format, not the content..
                    The vocabulary should be restricted to N3 level, use the vocabulary in the `Dictionary` as much as you can.
                    Please refer to the question examples following the formal exam paper.
                    Append the correct answer and explanation of the main challenges on why the teacher asks this question to the candidate in simplified Chinese at each question.
                    Finally, output beautiful markdown format.
                    Dictionary: {vocab_dict}
                    Search result: {search_result}
                    Formal exam paper: {example}
                """
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )

    
    input = { "topic" : state['topic'],
              "search_result": search_result, 
              "vocab_dict": vocab_dict,
              "example": example,
              "messages": state["messages"]
              }
    # final_message = prompt.format_messages(**input)
    # print(final_message)
    
    generate = prompt | azure_llm
    
    msg = generate.invoke(input=input)
    
    
    return {"question": msg.content, "messages": [AIMessage(content=msg.content)] }


def reflection_node(state: QuestionState) -> QuestionState:
    print("---REVISOR---")
    
    # Other messages we need to adjust
    cls_map = {"ai": HumanMessage, "human": AIMessage}
    # First message is the original user request. We hold it the same for all nodes
    translated = [state["messages"][0]] + [
        cls_map[msg.type](content=msg.content) for msg in state["messages"][1:]
    ]

    reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """you are a Japanese language educator reviewing a JLPT exam paper. Generate critique and recommendations for the user's submission.
            the review focuses on content accuracy and question quality. 
            - For content accuracy, you must verify that the grammar and vocabulary questions accurately reflect the appropriate JLPT N3 level, ensuring the reading passages are clear, relevant, and appropriately challenging. 
            - For question quality, you must ensure all questions are clearly worded and free from ambiguity to comprehensively assess different language skills, and confirm that the difficulty level of the questions matches the intended JLPT N3 level.
            - During detailed refinement, you check the format and presentation of the paper, ensuring it is well-organized and the instructions are clear and concise. you also ensure the content is culturally appropriate and relevant to Japanese language and culture.
            - Finally, you make give feedback, providing detailed recommendations, including requests.If you think the exam paper is good enough, you just say "GOOD ENOUGH"
            """
        ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    reflect = reflection_prompt | azure_llm
    
    res = reflect.invoke(translated)
    
    print(res.content)
    
    # We treat the output of this as human feedback for the generator
    return {"messages": [HumanMessage(content=res.content)]}



In [78]:
# Build workflow
builder = StateGraph(QuestionState)

builder.add_node("online_search", online_search)
builder.add_node("generator", question_draft_generator)
builder.add_node("reflector", reflection_node)
# Add nodes

def should_continue(state: QuestionState):
    if state["messages"]:
        if len(state["messages"]) > 6: 
            print("--- Reach the Maximum Round ---")
            return END
        elif "GOOD ENOUGH" in state["messages"][-1].content:
            print("--- AI Reviser feels Good Enough ---")
            return END
    return "generator"

# Add edges to connect nodes
builder.add_edge(START, "online_search")
builder.add_edge("online_search", "generator")
builder.add_edge("generator","reflector")
# 
builder.add_conditional_edges("reflector", should_continue)
memory = MemorySaver()

# Compile
kanji_graph = builder.compile()

# Show workflow
# display(Image(kanji_graph.get_graph().draw_png()))

In [79]:
row = words.iloc[1]
word = f"{row[0]}({row[1]})"
word

C:\Users\HW-BK003\AppData\Local\Temp\ipykernel_2376\846002302.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  word = f"{row[0]}({row[1]})"


'極(ごく)'

In [80]:
# # Debug the Conversation
# for event in kanji_graph.stream(
#     {
#         "messages": [
#             HumanMessage(
#                 content=word
#             )
#         ],
#     },
#     config={"configurable": {"thread_id": "1"}},
# ):
#     print(event)
#     print("---")

In [81]:
kanji = kanji_graph.invoke(
    {
       "messages": [
                HumanMessage(
                    content=word
                )
            ],
        },
    config={"configurable": {"thread_id": "1"}}
)
display(Markdown(kanji["question"]))

---WEB SEARCH---
{'query': '極(ごく)', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': '極 (ゴク)とは？ 意味や使い方 - コトバンク', 'url': 'https://kotobank.jp/word/極-53196', 'content': 'ごく【極】 [ 1 ] 〘 名詞 〙 ① ( 形動 ) 最上のもの。最良のもの。また、そのさま。極上。 [初出の実例]「『おれが面を立てにゃアおかねえ』トきっとこなし、蝙蝠安うなづいて、『違ひなし、そいつが極だ』」 (出典：歌舞伎・与話情浮名横櫛（1853）四幕) 「甘露煮にするにはこの位がごくだあ', 'score': 0.85157084, 'raw_content': None}, {'title': 'Meaning of 極, ごく, goku | Japanese Dictionary | JLearn.net', 'url': 'https://jlearn.net/dictionary/極', 'content': 'The english translations and meanings for 極, ごく and goku are: quite,very,10^48,quindecillion', 'score': 0.6989468, 'raw_content': None}, {'title': '極 - Wiktionary, the free dictionary', 'url': 'https://en.wiktionary.org/wiki/極', 'content': '極 きょく 地 ち (kyokuchi) 極 きょく 東 とう (Kyokutō) 極 ごく 道 どう (gokudō) 極 ごく 秘 ひ (gokuhi) 極 ごく 太 ぶと (gokubuto) 極 ごく 楽 らく (gokuraku) 極 きょく 論 ろん (kyokuron) 磁 じ 極 きょく (jikyoku) 電 でん 極 きょく (denkyoku) 極 ごく 悪 あく (gokuaku)', 'score': 0.644248, 'raw_content

### 聴解問題

#### 問題: 次の会話を聞いて、質問の意味に合う答えを選びなさい。

---

**会話内容:**  
- 女: 田中さん、今日の会議、どうでしたか?  
- 男: とても重要な内容でした。特に新しいプロジェクトについての話がごく重要でしたね。  
- 女: そうなんですね。じゃあ、次は何をする予定ですか?  
- 男: 今から、プロジェクトの準備を進めます。  

---

**質問:** 男の人は次に何をする予定ですか?  

1. 会議の内容を確認する  
2. 新しいプロジェクトの準備を進める  
3. 会議に再び参加する  

---

**正解:** 2  
**解説:**  
「ごく重要」という言葉を使うことで、新しいプロジェクトが非常に大切であることを強調しています。そのため、会話の流れから、男の人は次にプロジェクトの準備を進める予定であることがわかります。

**主な挑戦点:**  
この問題では、「ごく重要」という表現が出てきますが、これが「非常に重要」という意味であることを理解する必要があります。また、会話の流れをしっかりつかむことで、次の行動を正確に予測する能力が試されています。

--- 

**Markdown形式:**  
```markdown
### 聴解問題

#### 問題: 次の会話を聞いて、質問の意味に合う答えを選びなさい。

---

**会話内容:**  
- 女: 田中さん、今日の会議、どうでしたか?  
- 男: とても重要な内容でした。特に新しいプロジェクトについての話がごく重要でしたね。  
- 女: そうなんですね。じゃあ、次は何をする予定ですか?  
- 男: 今から、プロジェクトの準備を進めます。  

---

**質問:** 男の人は次に何をする予定ですか?  

1. 会議の内容を確認する  
2. 新しいプロジェクトの準備を進める  
3. 会議に再び参加する  

---

**正解:** 2  
**解説:**  
「ごく重要」という言葉を使うことで、新しいプロジェクトが非常に大切であることを強調しています。そのため、会話の流れから、男の人は次にプロジェクトの準備を進める予定であることがわかります。

**主な挑戦点:**  
この問題では、「ごく重要」という表現が出てきますが、これが「非常に重要」という意味であることを理解する必要があります。また、会話の流れをしっかりつかむことで、次の行動を正確に予測する能力が試されています。
```